<a href="https://colab.research.google.com/github/souhila1998/HGR_CNN-LSTM-SVM/blob/main/three_inputs_CNN_LSTM_SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Import dependencies
 

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from skimage.io import imread
from matplotlib import pyplot as plt
%matplotlib inline
import tensorflow 
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras import optimizers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Dense, Flatten,Dropout, Conv2D, MaxPooling2D, Activation, concatenate,LSTM, TimeDistributed
import datetime
import tensorflow_addons as tfa
from sklearn.model_selection import train_test_split
from yellowbrick.classifier import ClassPredictionError,ConfusionMatrix
import optuna
import time
import scikitplot as skplt
from tensorflow.keras.models import load_model
import cv2
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from yellowbrick.classifier import ClassificationReport

### Read labels and data


In [ ]:
train_df = pd.read_excel ('/labels.xlsx', index_col=0).apply(np.int64)
train_df = train_df.replace(-9223372036854775808, 0, regex=True)
train_df['left_file'] = train_df.index.map(lambda id: f'/x/{id}_x.png')
train_df['top_file'] = train_df.index.map(lambda id: f'/xy/{id}_xy.png')
train_df['right_file'] = train_df.index.map(lambda id: f'/y/{id}_y.png')

print (train_df)

### Parameters

In [ ]:
in_channel =1 #number of channel of the binary images
img_rows, img_cols = 75, 75 #image size
num_classes = 12 # number of classes
batch_size = 16

input_shape = (img_rows, img_cols, in_channel)
input_img = Input(shape = input_shape)

In [ ]:
def read_spectograms(file_paths, img_rows, img_cols, channels):
  images = []
  
  for file_path in file_paths:
    images.append(imread(file_path))

  images = np.asarray(images,  dtype=np.float32)
  images =  images /np.max(images)
 

  images = images.reshape(images.shape[0], img_rows, img_cols, channels)
    
  return images

 

### gradient of the images in the x-direction 


In [ ]:
x_train_left = read_spectograms(train_df.left_file.values, img_rows, img_cols, in_channel)


### gradient magnitude images  


In [ ]:
x_train_top = read_spectograms(train_df.top_file.values, img_rows, img_cols, in_channel)

### gradient of the image in the y-direction 


In [ ]:
x_train_right = read_spectograms(train_df.right_file.values, img_rows, img_cols, in_channel)

### Labels

In [ ]:
labels = train_df.classe.values
#The model consists of three inputs, the samples are stacked, 
#so the train_test_split function splits on the same pair of images.
#The stacking happens on the 4th axis.
x_train_comp = np.stack((x_train_left, x_train_top, x_train_right), axis=4)


## Create and train the model


### Split in train and test batches

In [ ]:
#split the images on the same pair of images
x_train, x_test, y_train, y_test = train_test_split(x_train_comp, labels, test_size = 0.2, random_state=1)

# take them apart
x_train_left = x_train[:,:,:,:,0]
x_test_left = x_test[:,:,:,:,0]
#####################################
x_train_top = x_train[:,:,:,:,1]
x_test_top = x_test[:,:,:,:,1]
#####################################
x_train_right = x_train[:,:,:,:,2]
x_test_right = x_test[:,:,:,:,2]

### Create the feature extractor 3-input CNN-LSTM


In [ ]:
from tensorflow.keras import initializers

def create_convolution_layers(input_img):

  model = Conv2D(16, (4,4), padding='same',strides = (2,2), input_shape=input_shape)(input_img)
  model = tfa.layers.Maxout(16)(model)
  model = MaxPooling2D(pool_size=(2, 2))(model)
  model = Dropout(0.25)(model)

  model = Conv2D(32, (4,4), padding='same',strides = (2,2))(model)
  model = tfa.layers.Maxout(32)(model)
  model = MaxPooling2D(pool_size=(2, 2))(model)
  model = Dropout(0.25)(model)


  model = Conv2D(64, (4,4), padding='same',strides = (2,2))(model)
  model = tfa.layers.Maxout(64)(model)
  model = MaxPooling2D(pool_size=(2, 2))(model)
  model = Dropout(0.25)(model)

  return model


In [ ]:
left_input = Input(shape=input_shape)
left_model = create_convolution_layers(left_input)

top_input = Input(shape=input_shape)
top_model = create_convolution_layers(top_input)

right_input = Input(shape=input_shape)
right_model = create_convolution_layers(right_input)
conv = concatenate([left_model, top_model, right_model])
conv = TimeDistributed(LSTM(units =150,recurrent_dropout=0.2))(conv)
conv = Flatten(name='flatten')(conv)
output = Dense(12, activation='softmax')(conv)

model = Model(inputs=[left_input, top_input, right_input], outputs=[output])

opt = optimizers.Adam()
# Compile the model
model.compile(loss=tensorflow.keras.losses.sparse_categorical_crossentropy,
              optimizer=opt,
              metrics=['accuracy'])
model.summary()

In [ ]:
from tensorflow.keras.utils import plot_model
%matplotlib inline
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

### Train the model
 

In [ ]:
%%time

history = model.fit([x_train_left, x_train_top, x_train_right], y_train,
          batch_size=batch_size,
          epochs=25,
          verbose=1,
          shuffle=False)

In [ ]:
#model.save("model_cnn_lstm.h5")

In [ ]:
model = load_model('model_cnn_lstm.h5')

In [ ]:
def tr_plot(tr_data, start_epoch):

   

    tacc=tr_data.history['accuracy']
    tloss=tr_data.history['loss']

    Epoch_count=len(tacc)+ start_epoch
    Epochs=[]
    for i in range (start_epoch ,Epoch_count):
        Epochs.append(i+1)   
    index_loss=np.argmin(tloss)
    val_lowest=tloss[index_loss]
    index_acc=np.argmax(tacc)
    acc_highest=tacc[index_acc]
    plt.style.use('fivethirtyeight')
    sc_label='best epoch= '+ str(index_loss+1 +start_epoch)
    vc_label='best epoch= '+ str(index_acc + 1+ start_epoch)
    fig,axes=plt.subplots(nrows=1, ncols=2, figsize=(20,8))

    axes[0].plot (Epochs,tacc,'r',label= 'Training Accuracy')
    plt.tick_params(labelsize=18)
    axes[0].set_title('Training Accuracy')
    axes[0].set_xlabel('Epochs')
    axes[0].set_ylabel('Accuracy')
    axes[0].legend()

    axes[1].plot(Epochs,tloss, 'r', label='Training loss')
    axes[1].set_title('Training Loss')
    axes[1].set_xlabel('Epochs')
    axes[1].set_ylabel('Loss')
    axes[1].legend()
    plt.show()

In [ ]:
tr_plot(history,0)


In [ ]:
model_feat = Model(inputs=model.input,outputs=model.get_layer('flatten').output)

feat_train = model_feat.predict([x_train_left, x_train_top, x_train_right])



In [ ]:
start = time.time()
print("Optimizing the parameters")
def objective(trial):
    kernel=trial.suggest_categorical('kernel',['rbf','poly','linear','sigmoid'])
    c=trial.suggest_float("C",0.1,3,log=True)
    gamma=trial.suggest_categorical('gamma',['auto','scale'])
    degree=trial.suggest_int("degree",1,3,log=True)
    classifier_obj  =OneVsRestClassifier(SVC(kernel=kernel,gamma=gamma,C=c, degree=degree))
    score = cross_val_score(classifier_obj,feat_train,y_train, n_jobs=-1, cv=5)
    accuracy = score.mean()
    return accuracy

In [ ]:
start = time.time()
print("Optimizing the parameters")

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)


end = time.time()
duration = end - start
print ('\n Parameter Optimization took %0.2f seconds (%0.1f minutes)'%(duration, duration/60) )

print("\n\nBEST TRIAL : \n",study.best_trial) #get best trial

In [ ]:
print("\n\nBEST PARAMETERS : \n",study.best_params) #get best parameters


In [ ]:
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_slice
plot_optimization_history(study)


In [ ]:
plot_slice(study)


### Train the SVM classifier

In [ ]:

#Best parameters : kernel='rbf',gamma='auto',c = 1.7077226779041297, degree = 3)
svc = OneVsRestClassifier(SVC(probability=True, kernel='rbf',gamma='auto',C= 1.7077226779041297, degree = 3))

training_start = time.perf_counter()

svc.fit(feat_train,y_train)

train_acc = svc.score(feat_train,y_train)*100

training_end = time.perf_counter()

### Make prediction with SVM 

In [ ]:
prediction_start = time.perf_counter()

feat_test = model_feat.predict([x_test_left, x_test_top, x_test_right])

preds = svc.predict(feat_test)

prediction_end = time.perf_counter()

pred_acc = (preds == y_test).sum().astype(float) / len(preds)*100



### SVM train/prediction accuracy/time 

In [ ]:
svc_train_time = training_end-training_start
svc_prediction_time = prediction_end-prediction_start
print("SVM training accuracy is: %3.16f" % (train_acc))
print("SVM prediction accuracy is: %3.16f" % (pred_acc))
print("Time consumed for training: %4.5f seconds" % (svc_train_time))
print("Time consumed for prediction: %6.5f seconds" % (svc_prediction_time))

In [ ]:
Y_true = y_test 


### Confusion matrix 

In [ ]:
fig = plt.figure(figsize=(10,7))
classes = ['L-R swipe', 'R-L swipe', 'U-D swipe','D-U swipe','Diag-LR-UD swipe','Diag-LR-DU swipe','Diag-RL-UD swipe','Diag-RL-DU swipe','clockwise rotation','counterclockwise rotation','inward push','empty gesture']

cm = ConfusionMatrix(svc, classes = classes, percent = True)

cm.fit(feat_train,y_train)
cm.score(feat_test,Y_true)

cm.poof()


### Classification report 

In [ ]:

fig = plt.figure(figsize=(7,7))

visualizer= ClassificationReport(svc,
                         classes = classes,     
                         support=True,
                         is_fitted=True, 
                         title="ClassPredictionError Classification Report"
                         )
visualizer.fit(feat_train,y_train,axis=1)
visualizer.score(feat_test,Y_true)
g = visualizer.show()

In [ ]:
preds = svc.predict_proba(feat_test)


In [ ]:
skplt.metrics.plot_roc_curve(Y_true, preds,
                       title="ROC Curve", figsize=(12,6));

In [ ]:
skplt.metrics.plot_precision_recall_curve(Y_true, preds,
                       title="Precision-Recall Curve", figsize=(12,6));

In [ ]:
# 